In [ ]:
#@title Imports { form-width: "20%" }
# Some imports that might be useful one day
!pip install recordtype
from recordtype import recordtype
import numpy as np
import urllib.request
import re
import collections
import itertools
import heapq
import functools

def get_input(day):
  return urllib.request.urlopen(f'https://raw.githubusercontent.com/SantoSimone'
                                f'/Advent-of-Code/master/2020/input_files/input'
                                f'{day}.txt').read().decode('utf-8')

def get_input_as_lines(day):
  return get_input(day).split('\n')[:-1]

def parse_ints(text):
  return [int(x) for x in re.findall(r'\d+', text)]

# Day 1

In [ ]:
#@title Part 1 { form-width: "20%" }

def d1p1():
  expenses = parse_ints(get_input(day))
  for val1, val2 in itertools.combinations(expenses, r=2):
    if val1 + val2 == 2020: return val1, val2 

# 'And so it begins' 
# (for non-nerds: this is a LOTR reference timing 2:46:41 enjoy!)
day = 1
val1, val2 = d1p1()
print(f'The two entries that sum to 2020 are: {val1} and {val2}. Their product'
      f' is {val1*val2}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d1p2():
  expenses = parse_ints(get_input(day))
  for val1, val2, val3 in itertools.combinations(expenses, r=3):
    if val1 + val2 + val3 == 2020: return val1, val2, val3

val1, val2, val3 = d1p2()
print(f'The three entries that sum to 2020 are: {val1}, {val2} and {val3}. '
      f'Their product is {val1*val2*val3}')

# Day 2

In [ ]:
#@title Part 1 { form-width: "20%" }

def d2p1(inputs):
  def check_policy(line):
    least, most, ch, pw = re.match(r'(\d+)-(\d+) (\w): (\w+)', line).groups()
    return int(least) <= collections.Counter(pw)[ch] <= int(most)

  return sum([check_policy(line) for line in inputs])
  
# It's day 2
day = 2
valid = d2p1(get_input_as_lines(day))
print(f'# of valid passwords: {valid}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Policy has changed, so we will change our policy func
def d2p2(inputs):
  def check_policy(line):
    least, most, ch, pw = re.match(r'(\d+)-(\d+) (\w): (\w+)', line).groups()
    return (pw[int(least) - 1] == ch) ^ (pw[int(most) - 1] == ch)

  return sum([check_policy(line) for line in inputs])
  

valid = d2p2(get_input_as_lines(day))
print(f'# of valid passwords: {valid}')

# Day 3

In [ ]:
#@title Part 1 { form-width: "20%" }

def d3p1(lines, start, slope):
    def generator(start, slope, width):
        i = 0
        while True:
            yield start[0] + (slope[0] * i) % width, start[1] + slope[1] * i
            i += 1

    def create_map(lines):
        return np.array([
            [x == '#' for x in line]  # columns
            for line in lines  # rows
        ], dtype=np.int32)  # ones and zeros are always nice

    tree_map = create_map(lines)
    height, width = tree_map.shape
    sum = 0
    for c, r in generator(start, slope, width):
        if r >= height: return sum
        sum += tree_map[r, c]

# It's day 3
day = 3
start = (0, 0)
slope = (3, 1) 
trees = d3p1(get_input_as_lines(day), start, slope)
print(f'# of trees with first policy: {trees}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Not an hard request, we simply need to call previous func 5 times
def d3p2(lines, start, slopes):
  return np.prod([d3p1(lines, start, slope) for slope in slopes])
  
slopes = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
trees = d3p2(get_input_as_lines(day), start, slopes)
print(f'Multiplied values of trees encountered with 5 policies: {trees}')

# Day 4

In [ ]:
#@title Part 1 { form-width: "20%" }
def parse_input(lines):
    ret = []
    passport = {}
    for line in lines:
        if line == '':
            ret.append(passport)
            passport = {}
            continue
        parts = line.split()
        for p in parts:
            k, v = p.split(':')
            passport[k] = v
    ret.append(passport)

    return ret

def check_passport(passport, required_fields):
  return all(field in passport for field in required_fields)

def d4p1(lines):
  passports = parse_input(lines)
  return np.sum([check_passport(passport, required_fields) 
                   for passport in passports])

# It's day 4
day = 4
lines = get_input_as_lines(day)
required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
optional_fields = ['cid']
s = d4p1(lines)
print(f'Valid passports: {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Now we want also data validation
def data_validation(passport):
  valid = True
  try:
      num, unit = re.match(r'(\d+)(\w+)', passport['hgt']).groups()
      if unit == 'cm':
          valid = (valid and (150 <= int(num) <= 193))
      elif unit == 'in':
          valid = (valid and (59 <= int(num) <= 76))
      else:
          return False

      return valid and (1920 <= int(passport['byr']) <= 2002) \
              and (2010 <= int(passport['iyr']) <= 2020) \
              and (2020 <= int(passport['eyr']) <= 2030) \
              and passport['hcl'][0] == '#' and len(passport['hcl']) == 7 \
              and all('0' <= c <= '9' or 'a' <= c <= 'f' for c in passport['hcl'][1:]) \
              and passport['ecl'] in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'] \
              and int(passport['pid']) and len(passport['pid']) == 9
  except:
      return False

def d4p2(lines):
  passports = parse_input(lines)
  return np.sum([check_passport(passport, required_fields)
                 and data_validation(passport)
                 for passport in passports])
  

s = d4p2(lines)
print(f'Valid passports with valid data: {s}')

# Day 5

In [ ]:
#@title Part 1 { form-width: "20%" }

def binary_space(line, upper_char, lower_char):
  return int(line.replace(upper_char, '1').replace(lower_char, '0'), base=2)

def d5p1(lines):
  return [binary_space(seat[:7], 'B', 'F') * 8 
          + binary_space(seat[7:], 'R', 'L')
          for seat in lines]

# It's day 5
day = 5
lines = get_input_as_lines(day)
highest = max(d5p1(lines))
print(f'Highest seat ID: {highest}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def check_pre_post(id, ids):
  return id if id + 1 in ids and id - 1 in ids and id not in ids else False

def d5p2(lines):
  ids = d5p1(lines)
  return np.sum([check_pre_post(id, ids) for id in range(min(ids), max(ids))])

my_id = d5p2(lines)
print(f'My ID is: {my_id}')

# Day 6

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_input(lines):
    return [
            list(g) 
            for k, g in itertools.groupby(lines, lambda x: x != '') if k
           ]

def d6p1(lines):
    groups = parse_input(lines)
    return sum([
        len(set(''.join(group))) for group in groups
    ])

# It's day 5
day = 6
lines = get_input_as_lines(day)
s = d6p1(lines)
print(f'Number of total questions: {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d6p2(lines):
    groups = parse_input(lines)
    all_questions = [set(''.join(group)) for group in groups]
    return sum(
        [
            all(x in g for g in groups[i])
            for i, group_set in enumerate(all_questions)
            for x in group_set
        ]
    )

s = d6p2(lines)
print(f'Number of total questions: {s}')

# Day 7

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_line(line):
  parts = re.split('bag', line)
  stuff = {
      key.strip(): int(value)
      for p in parts[1:]
      for value, key in re.findall(r'(\d+) ([a-z ]+)', p)
  }
  return {parts[0].strip(): stuff}

def parse_input(lines):
  bags = {}
  for line in lines:
      bags.update(parse_line(line))
  return bags

def rec_contains(bags, bag, bag_to_place):
  return bag_to_place == bag \
          or any(rec_contains(bags, inside_bag, bag_to_place) 
                 for inside_bag in bags[bag])

def d7p1(lines, bag_to_place):
  bags = parse_input(lines)
  return [rec_contains(bags, bag, bag_to_place) for bag in bags]

# It's day 7
day = 7
lines = get_input_as_lines(day)
# We sub 1 because shiny gold is counted in our algorithm
s = sum(d7p1(lines, 'shiny gold')) - 1
print(f'Shiny gold bag is contained in : {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def count_bags_inside(bags, outer_bag):
  # When we reach an empty bag we count 1 and we always add 1 to the count 
  # of inner bags cause we also count current outer bag
  return 1 + sum([num * count_bags_inside(bags, inner) 
                  for inner, num in bags[outer_bag].items()])


def d7p2(lines, bag_to_place):
  return count_bags_inside(parse_input(lines), bag_to_place)

# We sub 1 because shiny gold is counted in our algorithm
s = d7p2(lines, 'shiny gold') - 1
print(f'Shiny gold contains {s} bags')

# Day 8

In [ ]:
#@title Part 1 { form-width: "20%" }

Instruction = collections.namedtuple('Instruction', ['code', 'value'])

def parse_instructions(lines):
  return [
          Instruction(code=inst, value=int(value)) 
          for inst, value in [line.split() for line in lines]
          ]

def execute(inst, idx, acc):
  if inst.code == 'acc': return idx + 1, acc + inst.value
  elif inst.code == 'jmp': return idx + inst.value, acc
  elif inst.code == 'nop': return idx + 1, acc

def d8p1(instructions):
  i, accumulator, done = 0, 0, set()
  while i not in done:
    done = done.union({i})
    i, accumulator = execute(instructions[i], i, accumulator)

  return accumulator

# It's day 8
day = 8
instructions = parse_instructions(get_input_as_lines(day))
acc = d8p1(instructions)
print(f'Right before repeating an instruction the accumulator is: {acc}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def changed_instructions(instructions):
  changes = {'nop': 'jmp', 'jmp': 'nop'}
  for i, inst in enumerate(instructions):
    if inst.code in changes.keys():
      yield instructions[:i] \
            + [Instruction(code=changes[inst.code], value=inst.value)] \
            + instructions[i+1:] 

def d8p2(instructions):
  for changed in changed_instructions(instructions):
      i, accumulator, done = 0, 0, set()
      while i <= len(changed) - 1 and i not in done:
          done = done.union({i})
          i, accumulator = execute(changed[i], i, accumulator)
      if i >= len(changed): return accumulator

instructions = parse_instructions(get_input_as_lines(day))
acc = d8p2(instructions)
print(f'Changing one single operation the program terminates with: {acc}')

# Day 9

In [ ]:
#@title Part 1 { form-width: "20%" }

def valid(next, previous):
  prev_set = set(previous)
  if next not in map(lambda x: x[0] + x[1], itertools.combinations(prev_set, 2)):
    return False
  return True

def d9p1(nums, preamble):
    for i, num in enumerate(nums[preamble:]):
        if not valid(num, nums[i:i+preamble]): return num
    return -np.inf

# It's day 9
day = 9
nums = parse_ints(get_input(day))
invalid = d9p1(nums, 25)
print(f'First number that does not respect the policy is: {invalid}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d9p2(nums, preamble):
  invalid = d9p1(nums, preamble)
  start, end, partial = 0, 0, 0
  while partial != invalid:
      if partial > invalid:
          partial -= nums[start]
          start += 1
          continue
      else:
          partial += nums[end]
      end += 1
  return min(nums[start:end]) + max(nums[start:end])

weakness = d9p2(nums, 25)
print(f'The encryption weakness is: {weakness}')

# Day 10

In [ ]:
#@title Part 1 { form-width: "20%" }

def jolt_diff(low, high, condition):
  return 1 if (high - low) == condition else 0

def d10p1(adapters):
  sorted_adapters = sorted(adapters)
  subsequent = [(0, sorted_adapters[0])]  # first connection
  subsequent += [
                 (sorted_adapters[i], sorted_adapters[i + 1])
                 for i, _ in enumerate(sorted_adapters[:-1])
                ]
  ones = sum(map(lambda x: jolt_diff(x[0], x[1], 1), subsequent))
  threes = sum(map(lambda x: jolt_diff(x[0], x[1], 3), subsequent)) + 1
  return ones * threes

# It's day 10
day = 10
adapters = parse_ints(get_input(day))
ret = d10p1(adapters)
print(f'Multiplication of 1-jolts by 3-jolts is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }
# This was a though one: did not understand the stack of calls could be so
# overwhelming in a brute-force approach..
# Had to switch from a for-loop (A* search) to recursion (lru_cache friendly) 

State = collections.namedtuple('State', 'jolts, adapters')

def d10p2(adapters):
    # Add last connection
    adapters = sorted(adapters)
    goal = max(adapters) + 3  # final connection is built-in adapter
    adapters.append(goal)
    return recursive_search(State(0, frozenset(adapters)), goal)


@functools.lru_cache(maxsize=500)
def recursive_search(curr, goal):
    if curr.jolts == goal: return 1
    possible_nexts = possible_connections(curr)
    if not possible_nexts: return 0
    return sum([
        recursive_search(next_adap, goal) for next_adap in possible_nexts
    ])


def possible_connections(state):
    possibles = filter(lambda x: state.jolts < x <= state.jolts + 3, state.adapters)
    return [State(adap, state.adapters.difference({a for a in state.adapters if a <= adap})) for adap in possibles]

adapters = parse_ints(get_input(day))
ret = d10p2(adapters)
print(f'Number of possible configurations is: {ret}')

# Day 11

In [ ]:
#@title Part 1 { form-width: "20%" }

def create_seats_map(lines):
  # Seats will be: -1 floor, 0 empty, 1 occupied
  seats = [line.replace('.', '0').replace('L', '1') for line in lines]
  seats = [[int(s) for s in line] for line in seats]
  seats = np.array(seats) - 1
  return seats

def num_adjacent_occupied(seats, r, c):
  rows = [max(0, r - 1), r, min(r + 1, seats.shape[0] - 1)]
  cols = [max(0, c - 1), c, min(c + 1, seats.shape[1] - 1)]
  adjacent = set(itertools.product(rows, cols))
  # We do not count our seat, so we remove it from combinations
  adjacent.remove((r,c))
  return sum([1 for row, col in adjacent if seats[row, col] == 1])

def round_seat(seats, max_adjacent):
  def evaluate_single(seat, r, c):
    if seat == -1: return -1
    num_adjacent = num_adjacent_occupied(seats, r, c)
    
    # Empty and no occupied adjacent -> occupied
    if seat == 0 and num_adjacent == 0: return 1
    elif seat == 0: return 0
    
    # Occupied and `max_adjacent` or more adjacent occupied -> empty
    if seat == 1 and num_adjacent >= max_adjacent: return 0
    else: return 1

  new_seats = np.array(
      [evaluate_single(s, r, c)
       for (r, c), s in np.ndenumerate(seats)
      ]
  )
  # Above operation outputs the flattened matrix, so we reshape
  return np.reshape(new_seats, (seats.shape[0], seats.shape[1]))


def d11p1(lines, max_adjacent):
  last_seats = create_seats_map(lines)
  while True:
    new_seats = round_seat(last_seats, max_adjacent)
    if all((new_seats == last_seats).reshape(-1)):
      return sum(new_seats[new_seats > 0])
    last_seats = new_seats[:, :]

# It's day 11
day = 11
lines = get_input_as_lines(day)
ret = d11p1(lines, 4)
print(f'Final number of occupied seats is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# We only need to change the policies
def num_adjacent_occupied(seats, r, c):
    def check_direction(r, c, direction):
        row_dx, col_dx = direction
        h, w = seats.shape
        for i in range(1, max(h, w)):
            if not (-1 < r + row_dx * i < h and -1 < c + col_dx * i < w): return 0
            if seats[r + row_dx * i, c + col_dx * i] == -1:
                continue
            else:
                return seats[r + row_dx * i, c + col_dx * i]
        return 0

    directions = [(-1, -1), (-1, 0), (-1, 1),  # Topleft - Top - Topright
                  (0, -1), (0, 1),  # Left - Right
                  (1, -1), (1, 0), (1, 1)]  # Bottomleft - Bottom - Bottomright

    return sum([check_direction(r, c, dir) for dir in directions])

# It's day 11
day = 11
lines = get_input_as_lines(day)
ret = d11p1(lines, 5)
print(f'Final number of occupied seats is: {ret}')

# Day 12

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_instructions(input_txt):
    return [(code, int(value)) for (code, value) in re.findall(r'(\w)(\d+)', input_txt)]

def d12p1(input_txt):
    instructions = parse_instructions(input_txt)
    # East, North, West, South
    directions = np.array([(1, 0), (0, 1), (-1, 0), (0, -1)])
    direction_idx = 0
    pos = np.array([0, 0])
    for code, value in instructions:
        if code == 'N': pos += (0, value)
        if code == 'S': pos += (0, -value)
        if code == 'E': pos += (value, 0)
        if code == 'W': pos += (-value, 0)
        if code == 'L': direction_idx += (value // 90)
        if code == 'R': direction_idx -= (value // 90)
        if code == 'F': pos += directions[direction_idx % 4] * value

    return sum(np.abs(pos))

# It's day 12
day = 12
input_txt = get_input(day)
ret = d12p1(input_txt)
print(f'Manhattan distance from starting position (0,0) is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }
# Wanted to give a little bit of structure to ease the read of rotation
Position = recordtype('Position', 'x y')

def d12p2(input_txt):
  instructions = parse_instructions(input_txt)
  waypoint_pos = Position(x=10, y=1)
  ship_pos = Position(x=0, y=0)
  for code, value in instructions:
    if code == 'N': waypoint_pos.y += value
    elif code == 'S': waypoint_pos.y -= value
    elif code == 'E': waypoint_pos.x += value
    elif code == 'W': waypoint_pos.x -= value
    elif code == 'F':
        ship_pos.x += waypoint_pos.x * value
        ship_pos.y += waypoint_pos.y * value
    else:  # code is L or R now
      value = -value if code == 'L' else value
      # Cannot format decently, so i will keep this oneliners
      cos_v, sin_v = int(np.cos(np.deg2rad(value))), int(np.sin(np.deg2rad(value)))
      waypoint_pos.x, waypoint_pos.y = waypoint_pos.x * cos_v + waypoint_pos.y * sin_v, -waypoint_pos.x * sin_v + waypoint_pos.y * cos_v

  return np.abs(ship_pos.x) + np.abs(ship_pos.y)

ret = d12p2(input_txt)
print(f'Manhattan distance from starting position (0,0) is: {ret}')